In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sc
from sklearn import neighbors
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [43]:
def predictSales(df,model,test,stores,months):
    for store in stores:
        temp = test.loc[test['StoreID'] == store]
        for month in months:
            X = temp.loc[temp[month] == 1]
            X = X.drop(labels='StoreID', axis=1)
            res = model.predict(X)
            res = np.expm1(res)
            res = {'StoreID' : [store], 'Month' : [month], 'NumberOfSales' : [np.sum(res)]}
            res = pd.DataFrame(res)
            df = df.append(res,ignore_index=True)
    
    df.to_csv("Results.csv")

In [17]:

# Need train set, index of varibales, index of target, test size and seed

def randomSplit(dataset, variable,target,test_size=0.33, random_state=1234):

    train_data, test_data = model_selection.train_test_split(dataset, test_size=0.33, random_state=1234)


    train_x = train_data[variable].values
    train_y = train_data[target].values

    test_x = test_data[variable].values
    test_y = test_data[target].values

#    train_x = train_x.reshape(len(train_x), degree)
    train_y = train_y.reshape(len(train_y), 1)
#    test_x = test_x.reshape(len(test_y), degree)
    test_y = test_y.reshape(len(test_y), 1)


    return train_x,train_y,test_x,test_y

In [45]:
#stores = test['StoreID'].unique()
#months = ['March','April']
#df = pd.DataFrame(columns=['StoreID', 'Month', 'NumberOfSales'])
#predictSales(df,knn,test,stores,months)

# K Nearest Neighbors 

In [46]:
train = pd.read_csv('TrainClean.csv',index_col=0)
test = pd.read_csv('TestClean.csv',index_col=0)

In [47]:
target = train['NumberOfSales']
variables = train.drop(labels='NumberOfSales',axis=1)

In [48]:
train['NumberOfSales'].describe()

count    433958.000000
mean          2.239679
std           0.050325
min           0.000000
25%           2.211314
50%           2.241269
75%           2.270551
max           2.415042
Name: NumberOfSales, dtype: float64

In [49]:
#target = np.expm1(target.values)

In [50]:
#sc.describe(target)

In [51]:
var = train.columns.drop('NumberOfSales')
tar = 'NumberOfSales'

In [52]:
RMSE = {}
R2 = {}

for ii in range(2,5):
    
    tr_x,tr_y,tes_x,tes_y = randomSplit(train,var,tar)
    
    n_neighbors = ii
    knn = neighbors.KNeighborsRegressor(n_neighbors)
    knn = knn.fit(tr_x, tr_y)

    yp = knn.predict(tes_x)
    yt = tes_y

    RMSE[ii]=np.sqrt (sum((yt - yp)*(yt - yp)) / len(yt))
    R2[ii]=knn.score(tes_x,tes_y)

    print("TEST RESULTS ->RMSE : "+str(RMSE[ii])+" R2 :"+str(R2[ii]) +" for KNN with "+str(n_neighbors)+" neighbors")

TEST RESULTS ->RMSE : [0.04291793] R2 :0.21128278961651506 for KNN with 2 neighbors
TEST RESULTS ->RMSE : [0.03966811] R2 :0.3262065645045549 for KNN with 3 neighbors
TEST RESULTS ->RMSE : [0.03804062] R2 :0.3803606164800103 for KNN with 4 neighbors


In [10]:
#scores = cross_val_score(knn, variables, target, cv=KFold(n_splits=10, random_state=1234, shuffle=True))

# Random Forset

In [12]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sc
from sklearn import neighbors
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [13]:
train = pd.read_csv('TrainClean.csv',index_col=0)
test = pd.read_csv('TestClean.csv',index_col=0)

In [14]:
target = train['NumberOfSales']
variables = train.drop(labels='NumberOfSales',axis=1)

In [15]:
ensemble_clfs = [
    ("RandomForestRegressor, max_features='sqrt'",
        RandomForestRegressor(oob_score=True,
                               max_features="sqrt",
                               random_state=1234)),
    ("RandomForestRegressor, max_features='log2'",
        RandomForestRegressor(max_features='log2', oob_score=True,
                               random_state=1234)),
    ("RandomForestRegressor, max_features=None",
        RandomForestRegressor(max_features=None, oob_score=True,
                               random_state=1234))
]
# Warm_start = When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble, otherwise, just fit a whole new forest.

In [ ]:
estimators = 100
OOB_Scores = {}

for label, clf in ensemble_clfs:
        clf.set_params(n_estimators=estimators)
        clf.fit(variables, target)
        OOB_Scores[(label,clf)] = oob_error = 1 - clf.oob_score_